In [1]:
import numpy as np
import tensorflow as tf
import t3f
tf.set_random_seed(0)
np.random.seed(0)
%matplotlib inline
import matplotlib.pyplot as plt
import metric_util as mt
import data_util as du
from t3f import shapes
from nilearn import image
import nibabel as nib
import math
from math import sqrt
import metric_util
import mri_draw_utils as mrd
import matplotlib.gridspec as gridspec
from nilearn import plotting
import tensor_util as tu



SyntaxError: non-keyword arg after keyword arg (mri_draw_utils.py, line 964)

In [ ]:
import spike_detection as sp

In [ ]:
subject_scan_path = du.get_full_path_subject1()

In [ ]:
spike_thre = 4

In [ ]:
title = "Slise-wise 4D fMRI Spikes in a Fourier Space at the extremes above Z-Score " + '$ >$' + str(spike_thre)

In [ ]:
mrd.draw_spikes(subject_scan_path, title, "/work/str/fft", spike_thre = 4.)

In [ ]:
def plot_slice_tern(in_fft, tr, z_score, outer_grid, fig, z,
                  cmap='Greys_r'):
    
    inner_grid = gridspec.GridSpecFromSubplotSpec(1, 3,
            subplot_spec=outer_grid, wspace=0.0, hspace=0.0)
    
    main_ax = fig.add_subplot(inner_grid[0, 0])
    main_ax.set_aspect('equal')
    
    fg_color = 'white'
    bg_color = 'black'
    
    title = "TR = " + str(tr) + " z = " + str(z)
    main_ax.set_title(title, color=fg_color, fontweight='normal', fontsize=8)  
    
    prev_spike_tr_img, prev_spike_zscored_overlay_img = mrd.get_prev_tr_img_with_overlay(in_fft, tr, z_score)
    
    if  prev_spike_tr_img is not None:
        prev_spike_img = plotting.plot_img( prev_spike_tr_img, figure= fig, axes = main_ax, display_mode='z', 
                                            bg_img=None,black_bg=True, 
                                            cmap=cmap, cut_coords=[z])
        prev_spike_img.add_contours(prev_spike_zscored_overlay_img, 
                                    levels=[0.5], filled=True, alpha=0.8, colors='r')
        
    curr_tr_ax = fig.add_subplot(inner_grid[0, 1], sharey=main_ax)
    
    curr_tr_spike_tr_img, curr_tr_spike_zscored_overlay_img = mrd.get_spiked_tr_img_with_overlay(in_fft, tr, z_score)
    
    curr_img = plotting.plot_img(curr_tr_spike_tr_img, figure= fig, axes = curr_tr_ax, display_mode='z', 
                                            bg_img=None,black_bg=True, 
                                            cmap=cmap, cut_coords=[z])
    
    curr_img.add_contours(curr_tr_spike_zscored_overlay_img, 
                                    levels=[0.5], filled=True, alpha=0.8, colors='r')
    
    post_tr_ax = fig.add_subplot(inner_grid[0, 2], sharey=main_ax)
    
    post_spike_tr_img, post_spike_zscored_overlay_img = mrd.get_post_tr_img_with_overlay(in_fft, tr, z_score)
    
    if  post_spike_tr_img is not None:
        post_spike_img = plotting.plot_img( post_spike_tr_img, figure= fig, axes = post_tr_ax, display_mode='z', 
                                            bg_img=None,black_bg=True, 
                                            cmap=cmap, cut_coords=[z])
        post_spike_img.add_contours(post_spike_zscored_overlay_img, 
                                    levels=[0.5], filled=True, alpha=0.8, colors='r')
    

def draw_spikes(path_func, plot_title, folder, spike_thre = 4.):
    n_spikes, out_spikes, out_fft, spikes_list = sp.slice_wise_fft(path_func, folder, spike_thres=spike_thre, out_prefix = 'subject')

    fig = plt.figure(frameon = False, figsize=(10,10))
    fig.set_size_inches(7, 7)
    cols=3
    
    fg_color = 'white'
    bg_color = 'black'
    
    if plot_title:
        fig.suptitle(plot_title, color=fg_color, fontweight='normal', fontsize=11)
    
    if len(spikes_list) > cols * 7:
        cols += 1
        
    nspikes = len(spikes_list)
    rows = 1
    if nspikes > cols:
        rows = math.ceil(nspikes / cols)
        
    print cols, rows
    fig = plt.figure(figsize=(int(7 * cols), int(5 * rows)))
    outer_grid = gridspec.GridSpec(int(rows), int(cols), hspace=0.2, wspace=0.2)
    
    for i, (t, z) in enumerate(spikes_list):
            subplot_spec = outer_grid[i]
            print i
            plot_slice_tern(out_fft, t, spike_thre, subplot_spec, fig, z,
                  cmap='Greys_r')

In [ ]:
draw_spikes(subject_scan_path, "Test", None, spike_thre = 4.)

In [ ]:
n_spikes, out_spikes, out_fft, spikes_list = sp.slice_wise_fft(subject_scan_path, None, spike_thres=4.0, out_prefix = 'subject')

In [ ]:
prev_spike_tr_img, prev_spike_zscored_overlay_img = mrd.get_prev_tr_img_with_overlay(out_fft, 39, 4.0)

In [ ]:
print prev_spike_tr_img

In [ ]:
print prev_spike_zscored_overlay_img

In [ ]:
def get_post_tr_img_with_overlay(in_fft, tr, z_score):
    data = np.array(in_fft.get_data())
    
    post_spike_tr_img = None
    post_spike_zscored_overlay_img = None
    
    ntpoints = data.shape[-1]
    if tr < (ntpoints - 1):
        post_spike_tr_img, post_spike_zscored_overlay_img = get_spiked_tr_img_with_overlay(in_fft, tr + 1, z_score)

In [ ]:
get_post_tr_img_with_overlay(out_fft, 39, 4.0)

In [ ]:
print out_fft